# Imports

In [1]:
!pip install clean-text
!pip install protobuf==4.25.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.1 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171031 sha256=c4d51474d39a0f471721f6a81ffb3ba226287ffdbea08e77a52405137b42109b
  Stored in directory: /root/.cache/pip/wheels/bd/22/e5/b69726d5e1a19795ecd3b3e7464b16c0f1d019aa94ff1c8578
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.15.0
    Uninstalling emoji-2.15.0:
      Successfully uninstalled emoji-2.15.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

In [2]:
# Pytorch related
import torch
import torch.nn as nn

# sklearn and scipy related
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from scipy import stats

# Huggingface related
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForMaskedLM, 
    DataCollatorForLanguageModeling, 
    TrainingArguments, 
    Trainer, 
    AutoConfig, 
    AutoModelForSequenceClassification,
    EarlyStoppingCallback
)
from huggingface_hub import snapshot_download

# text preprocessing
from cleantext import clean
import re

# miscellanious
import math
import pandas as pd
import numpy as np
import json
import random
from tqdm import tqdm
import os

2025-11-07 07:14:51.098578: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762499691.296543      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762499691.353753      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Reddit Domain Adaptation

## Model

In [3]:
model_checkpoint = "roberta-base"
model_path = snapshot_download(repo_id=model_checkpoint)
model_path

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/445 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

flax_model.msgpack:   0%|          | 0.00/499M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.08k [00:00<?, ?B/s]

...

tf_model.h5:   0%|          | 0.00/657M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

'/root/.cache/huggingface/hub/models--roberta-base/snapshots/e2da8e2f811d1448a5b465c236feacd80ffbac7b'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForMaskedLM.from_pretrained(model_path)

In [5]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer

RobertaTokenizerFast(name_or_path='/root/.cache/huggingface/hub/models--roberta-base/snapshots/e2da8e2f811d1448a5b465c236feacd80ffbac7b', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
)

In [6]:
mask_token = tokenizer.mask_token
mask_token_id = tokenizer.mask_token_id
mask_token, mask_token_id

('<mask>', 50264)

In [7]:
inputs = tokenizer("I love [MASK] food", return_tensors="pt")
outputs = model(**inputs)
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
logits = outputs.logits[0, mask_token_index, :]
predicted_token_id = logits.argmax(dim=-1)
print(tokenizer.decode(predicted_token_id))

In [8]:
model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNor

## Dataset

In [9]:
dsr = load_dataset("Fredithefish/Reddit-TIFU")
dsr

tifu_collection.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/619 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'meta'],
        num_rows: 619
    })
})

In [10]:
dsr['train']['meta']

Column([{'source': 'Reddit'}, {'source': 'Reddit'}, {'source': 'Reddit'}, {'source': 'Reddit'}, {'source': 'Reddit'}])

In [11]:
dsr['train'][0]

{'title': 'TIFU by raising the flag upside down on a military base and causing local farmers to think the base was in distress.',
 'text': 'First things first, I am not, and never have been, in the military.  This story concerns fifteen year old me when I was a Sea Cadet, which is kind of like Boy Scouts but directly connected to the US Navy. We\'d wear the (at the time) blue camouflage US Navy uniforms, with a giant gold sign on our shirts and hats that made clear we weren\'t actually in the Navy.  Like us being literal children wasn\'t a clue enough...\n\nIn the summer and school breaks, we could go to camps.  You could go on an abbreviated Naval Diver course.  You could go to Seabee camp and learn a bit about combat engineering and shoot airsoft guns at each other in training. You could go on a course to learn to defensive drive Humvees, which I used as training hours for my learner\'s permit. I can\'t express in words how bizarre and amazing this group was when I was a kid. \n\nTo 

In [12]:
dsr['train']['text']

Column(['First things first, I am not, and never have been, in the military.  This story concerns fifteen year old me when I was a Sea Cadet, which is kind of like Boy Scouts but directly connected to the US Navy. We\'d wear the (at the time) blue camouflage US Navy uniforms, with a giant gold sign on our shirts and hats that made clear we weren\'t actually in the Navy.  Like us being literal children wasn\'t a clue enough...\n\nIn the summer and school breaks, we could go to camps.  You could go on an abbreviated Naval Diver course.  You could go to Seabee camp and learn a bit about combat engineering and shoot airsoft guns at each other in training. You could go on a course to learn to defensive drive Humvees, which I used as training hours for my learner\'s permit. I can\'t express in words how bizarre and amazing this group was when I was a kid. \n\nTo do any of these though, you had to go through Sea Cadet basic training camp, which is held at an Air National Guard base in Marseil

## Preprocessing

In [13]:
def preprocessing(ds):
    """
    Only removing urls and hashtags from the text.
    Why not lowercasing - because bold words in social media means the person is trying to shout or more focus is given there.
    Why not punctuation - punctuations have can alter the meaning of a sentence
    Why not emojis or emoticons - Social media or online texts have so much usage of them.
    """
    texts = ' '.join(ds['train']['text'])
    
    cleaned = clean(texts, lower=False, no_urls=True, no_punct=False, replace_with_url='')
    cleaned = re.sub(r'#\w+', '', cleaned)
    cleaned = ' '.join(cleaned.split())

    split_words = 128
    splits = cleaned.split()
    chunks = [' '.join(splits[i:i+split_words]) for i in range(0, len(splits), split_words)]

    return chunks

corpus = preprocessing(dsr)
len(corpus)

1991

In [14]:
print(type(corpus), len(corpus))
print(corpus[:3])

<class 'list'> 1991
["First things first, I am not, and never have been, in the military. This story concerns fifteen year old me when I was a Sea Cadet, which is kind of like Boy Scouts but directly connected to the US Navy. We'd wear the (at the time) blue camouflage US Navy uniforms, with a giant gold sign on our shirts and hats that made clear we weren't actually in the Navy. Like us being literal children wasn't a clue enough... In the summer and school breaks, we could go to camps. You could go on an abbreviated Naval Diver course. You could go to Seabee camp and learn a bit about combat engineering and shoot airsoft guns at each other in training. You could go on a course to", "learn to defensive drive Humvees, which I used as training hours for my learner's permit. I can't express in words how bizarre and amazing this group was when I was a kid. To do any of these though, you had to go through Sea Cadet basic training camp, which is held at an Air National Guard base in Marseil

In [15]:
ds = Dataset.from_dict({'text' : corpus})
ds = ds.train_test_split(test_size=0.2, seed=42)
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1592
    })
    test: Dataset({
        features: ['text'],
        num_rows: 399
    })
})

In [16]:
for i in range(3):
    print(ds['train'][i])

{'text': 'have a rather large house and can sleep about ten people comfortably. My bedroom is on the main fooor adjacent to the kitchen and the entertainment space. This will be relevant shortly. I had been told for months on end that my guests would be arriving Thursday morning. I had hired a maid to clean the entire house top to bottom on Tuesday with the exception of my bedroom which I prefer to clean myself. Cut to Wedensday morning before work. I was wanting to blow off some steam before the extended family arrive and had quite a morning with my extensive collection. I own seven Real Dolls with costumes ranging from Zero Suit Samus to Queen Maeve to Rey Skywalker. I cosplay dress all my real dolls.'}
{'text': "that meant I was into him and wanted to invite him into my home and let him slide into that 🤢 TL;DR - A guy knocked on my door asking for directions and I stupidly gave him my number trying to provide a side point of contact knowing how hard it is to stay sober when all your

## Guided Masking

In [17]:
!wget https://saifmohammad.com/WebDocs/Lexicons/NRC-Emotion-Lexicon.zip
!unzip NRC-Emotion-Lexicon.zip

--2025-11-07 07:15:27--  https://saifmohammad.com/WebDocs/Lexicons/NRC-Emotion-Lexicon.zip
Resolving saifmohammad.com (saifmohammad.com)... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


192.185.17.122
Connecting to saifmohammad.com (saifmohammad.com)|192.185.17.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25878449 (25M) [application/zip]
Saving to: ‘NRC-Emotion-Lexicon.zip’

NRC-Emotion-Lexicon 100%[===================>]  24.68M  9.73MB/s    in 2.5s    

2025-11-07 07:15:30 (9.73 MB/s) - ‘NRC-Emotion-Lexicon.zip’ saved [25878449/25878449]

Archive:  NRC-Emotion-Lexicon.zip
   creating: NRC-Emotion-Lexicon/
  inflating: NRC-Emotion-Lexicon/Paper-Practical-Ethical-Considerations-Lexicons.pdf  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._Paper-Practical-Ethical-Considerations-Lexicons.pdf  
  inflating: NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-ForVariousLanguages.txt  

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



  inflating: NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._NRC-Emotion-Lexicon-Wordlevel-v0.92.txt  
  inflating: NRC-Emotion-Lexicon/Paper1_NRC_Emotion_Lexicon.pdf  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._Paper1_NRC_Emotion_Lexicon.pdf  
  inflating: NRC-Emotion-Lexicon/README.txt  
  inflating: NRC-Emotion-Lexicon/Paper-Ethics-Sheet-Emotion-Recognition.pdf  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._Paper-Ethics-Sheet-Emotion-Recognition.pdf  
  inflating: NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Senselevel-v0.92.txt  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._NRC-Emotion-Lexicon-Senselevel-v0.92.txt  
   creating: NRC-Emotion-Lexicon/OneFilePerLanguage/
  inflating: NRC-Emotion-Lexicon/ListOfLanguages-For-Which-Lexicon-Availabale.txt  
  inflating: NRC-Emotion-Lexicon/Paper2_NRC_Emotion_Lexicon.pdf  
  inflating: __MACOSX/NRC-Emotion-Lexicon/._Paper2_NRC_Emotion_Lexicon.pdf  
   creating: NRC-Emotion-Lexicon/OneFil

In [18]:
filepath = '/kaggle/working/NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'
nrc = (
    pd.read_csv(filepath,
                names=["word", "emotion", "association"],
                sep='\t',
                skiprows=0,
                keep_default_na=False)
    .query("association == 1")
)
nrc

,word,emotion,association
19,abacus,trust,1
23,abandon,fear,1
25,abandon,negative,1
27,abandon,sadness,1
30,abandoned,anger,1
...,...,...,...
141461,zest,anticipation,1
141464,zest,joy,1
141466,zest,positive,1
141469,zest,trust,1


In [19]:
nrc['emotion'].unique()

array(['trust', 'fear', 'negative', 'sadness', 'anger', 'surprise',
       'positive', 'disgust', 'joy', 'anticipation'], dtype=object)

In [20]:
lexicon = nrc['word'].unique()
lexicon = set(map(lambda x : x.lower(), lexicon))
len(lexicon)

6453

In [21]:
def guided_masking(ds, mask_size=0.2, mask_token='[MASK]'):
    """
    Mask 20% of words in the text that appear in lexicon.
    If <20% masked, fill remainder with random masking.
    """
    text = ds["text"]
    words = re.findall(r"\w+|\S", text) 
    n_words = len([w for w in words if re.match(r"\w+", w)])
    
    if n_words == 0:
        ds["masked_text"] = text
        return ds
    
    # Mask positions where word is in lexicon
    maskable_indices = [i for i, w in enumerate(words) if w.lower() in lexicon]
    n_to_mask = max(1, int(mask_size * n_words))
    
    masked_indices = set(random.sample(maskable_indices, min(len(maskable_indices), n_to_mask)))
    
    # If not enough guided words, do random masking
    if len(masked_indices) < n_to_mask:
        remaining = n_to_mask - len(masked_indices)
        all_word_indices = [i for i, w in enumerate(words) if re.match(r"\w+", w)]
        available = list(set(all_word_indices) - masked_indices)
        random_indices = random.sample(available, min(remaining, len(available)))
        masked_indices.update(random_indices)

    masked_words = [mask_token if i in masked_indices else w for i, w in enumerate(words)]
    
    # Apply masking
    masked_text = ""
    for i, w in enumerate(masked_words):
        if i > 0 and re.match(r"\w+|"+re.escape(mask_token), w) and re.match(r"\w+|"+re.escape(mask_token), masked_words[i - 1]):
            masked_text += " "
        masked_text += w
    
    ds["masked_text"] = masked_text
    return ds

In [22]:
ds = ds.map(guided_masking, fn_kwargs={
    'mask_token' : tokenizer.mask_token
})
ds

Map:   0%|          | 0/1592 [00:00<?, ? examples/s]

Map:   0%|          | 0/399 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'masked_text'],
        num_rows: 1592
    })
    test: Dataset({
        features: ['text', 'masked_text'],
        num_rows: 399
    })
})

In [23]:
for i in range(3):
    print(ds['train'][i])

{'text': 'have a rather large house and can sleep about ten people comfortably. My bedroom is on the main fooor adjacent to the kitchen and the entertainment space. This will be relevant shortly. I had been told for months on end that my guests would be arriving Thursday morning. I had hired a maid to clean the entire house top to bottom on Tuesday with the exception of my bedroom which I prefer to clean myself. Cut to Wedensday morning before work. I was wanting to blow off some steam before the extended family arrive and had quite a morning with my extensive collection. I own seven Real Dolls with costumes ranging from Zero Suit Samus to Queen Maeve to Rey Skywalker. I cosplay dress all my real dolls.', 'masked_text': 'have a rather large house and can sleep about ten people comfortably.My bedroom is <mask> the <mask> fooor adjacent to <mask> kitchen and the <mask> space.<mask> will be <mask> <mask>.I had been told for months on end that my <mask> would be arriving Thursday morning.I

In [24]:
def tokenize_function(example):
    # Encode masked text as input, original text as label
    model_inputs = tokenizer(
        example["masked_text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

    labels = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs

In [25]:
tokenized_ds = ds.map(tokenize_function, batched=True, remove_columns=ds["train"].column_names)
tokenized_ds

Map:   0%|          | 0/1592 [00:00<?, ? examples/s]

Map:   0%|          | 0/399 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1592
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 399
    })
})

In [26]:
batch_size = 64 
model_name = model_checkpoint.split("/")[-1]
output_dir = f"{model_name}-domain-adapted"

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    eval_strategy="epoch",
    learning_rate=5e-5,  
    weight_decay=0.01,
    num_train_epochs=6, 
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False, 
    fp16=True,  
    save_steps=1000,  
    logging_steps=len(tokenized_ds["train"]) // (batch_size * 2), 
    report_to="none", 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    # data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipykernel_19/3033905897.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,3.269900,3.053230
2,2.781900,2.733220
3,2.651000,2.687483
4,2.576700,2.641482
5,2.470000,2.681317
6,2.364300,2.642933


TrainOutput(global_step=150, training_loss=2.905130246480306, metrics={'train_runtime': 162.8163, 'train_samples_per_second': 58.667, 'train_steps_per_second': 0.921, 'total_flos': 628677926350848.0, 'train_loss': 2.905130246480306, 'epoch': 6.0})

In [27]:
eval_results = trainer.evaluate()
perplexity = math.exp(eval_results["eval_loss"])
print(f"Perplexity: {perplexity:.2f}")

trainer.save_model() 

Perplexity: 14.05


## Testing

In [28]:
model_path = "/kaggle/working/roberta-base-domain-adapted" 
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForMaskedLM.from_pretrained(model_path)
model.eval()

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNor

In [29]:
def predict_masked_tokens(text, top_k=5):
    """
    Predict top-k tokens for each [MASK] in the text.
    """
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits

    masked_indices = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

    for idx in masked_indices:
        probs = predictions[0, idx].softmax(dim=0)
        topk = torch.topk(probs, k=top_k)
        tokens = tokenizer.convert_ids_to_tokens(topk.indices)
        scores = topk.values.tolist()

        print(f"\nMask position {idx.item()}:")
        for token, score in zip(tokens, scores):
            token = token.replace('Ġ', '')
            print(f"  {token:15s} | {score:.4f}")


In [30]:
text = f"I have been feeling very {tokenizer.mask_token} lately."
predict_masked_tokens(text)


Mask position 6:
  sick            | 0.2476
  tired           | 0.2149
  depressed       | 0.1024
  ill             | 0.0389
  anxious         | 0.0269


In [31]:
text = f"I find it hard to {tokenizer.mask_token} in activities I used to enjoy."
predict_masked_tokens(text)


Mask position 6:
  participate     | 0.6750
  engage          | 0.2114
  partake         | 0.0128
  rein            | 0.0116
  indulge         | 0.0103


In [32]:
base_model = AutoModelForMaskedLM.from_pretrained("roberta-base")

def compare_models(text):
    inputs = tokenizer(text, return_tensors="pt")

    with torch.no_grad():
        base_logits = base_model(**inputs).logits
        domain_logits = model(**inputs).logits

    # Find mask index
    mask_idx = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1][0]

    # Compute probabilities
    base_probs = base_logits[0, mask_idx].softmax(dim=0)
    domain_probs = domain_logits[0, mask_idx].softmax(dim=0)

    # Get top 5 predictions
    base_top = torch.topk(base_probs, 5)
    domain_top = torch.topk(domain_probs, 5)

    print("\nBase RoBERTa predictions:")
    for token_id, score in zip(base_top.indices, base_top.values):
        token = tokenizer.convert_ids_to_tokens(token_id.item())
        token = token.replace("Ġ", "")
        print(f"  {token:15s} | {score:.4f}")

    print("\nDomain-adapted RoBERTa predictions:")
    for token_id, score in zip(domain_top.indices, domain_top.values):
        token = tokenizer.convert_ids_to_tokens(token_id.item())
        token = token.replace("Ġ", "")
        print(f"  {token:15s} | {score:.4f}")

In [33]:
original = "I used to be able to cry"
text = f"I used to be able to {tokenizer.mask_token}."
compare_models(text)


Base RoBERTa predictions:
  read            | 0.1334
  fly             | 0.1102
  drive           | 0.0911
  talk            | 0.0555
  write           | 0.0439

Domain-adapted RoBERTa predictions:
  talk            | 0.1805
  read            | 0.1761
  write           | 0.0825
  drive           | 0.0699
  cook            | 0.0353


In [34]:
original = "I hate myself"
text = f"I {tokenizer.mask_token} myself"
compare_models(text)


Base RoBERTa predictions:
  hate            | 0.2003
  blame           | 0.0800
  forgive         | 0.0726
  love            | 0.0567
  doubt           | 0.0449

Domain-adapted RoBERTa predictions:
  hate            | 0.3221
  love            | 0.0944
  blame           | 0.0796
  doubt           | 0.0333
  forgive         | 0.0274


In [35]:
original = "I have thoughts of killing myself."
text = f"I have thoughts of {tokenizer.mask_token} myself."
compare_models(text)


Base RoBERTa predictions:
  killing         | 0.7077
  cutting         | 0.0273
  shooting        | 0.0242
  hurting         | 0.0205
  harming         | 0.0136

Domain-adapted RoBERTa predictions:
  killing         | 0.8393
  shooting        | 0.0452
  hurting         | 0.0218
  harming         | 0.0157
  poisoning       | 0.0111


# Mental Domain Adaptation

## Dataset

In [36]:
dsm1 = load_dataset("ShreyaR/DepressionDetection")
dsm1

depression_dataset_reddit_twitter.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/7731 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'is_depression'],
        num_rows: 7731
    })
})

In [37]:
dsm2 = load_dataset("hugginglearners/reddit-depression-cleaned")
dsm2

README.md: 0.00B [00:00, ?B/s]

depression_dataset_reddit_cleaned.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/7731 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'is_depression'],
        num_rows: 7731
    })
})

In [38]:
dsm3 = load_dataset("jsfactory/mental_health_reddit_posts")
dsm3

posts.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/24000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['body', 'subreddit'],
        num_rows: 24000
    })
})

In [39]:
dsm3['train'][0]

{'body': 'Thank you.\n\nMy son was recently diagnosed and I want to do whatever I can to support and help him. His current teacher also has ADHD and has been wonderful. I’m hopeful for my son.',
 'subreddit': 1}

In [40]:
merged_dsm = (
    list(dsm1["train"]["clean_text"]) +
    list(dsm2["train"]["clean_text"]) +
    list(dsm3["train"]["body"])
)

print(f"Total merged samples: {len(merged_dsm)}")

Total merged samples: 39462


## Preprocessing

In [41]:
def preprocessing(merged_dsm):
    """
    Same as above, just the input types is changed.
    """
    texts = merged_dsm
    
    cleaned = clean(texts, lower=False, no_urls=True, no_punct=False, replace_with_url='')
    cleaned = re.sub(r'#\w+', '', cleaned)
    cleaned = ' '.join(cleaned.split())

    split_words = 128
    splits = cleaned.split()
    chunks = [' '.join(splits[i:i+split_words]) for i in range(0, len(splits), split_words)]

    return chunks

corpus = preprocessing(merged_dsm)
len(corpus)

17991

In [42]:
ds = Dataset.from_dict({'text' : corpus})
ds = ds.train_test_split(test_size=0.2, seed=42)
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 14392
    })
    test: Dataset({
        features: ['text'],
        num_rows: 3599
    })
})

In [43]:
ds = ds.map(guided_masking, fn_kwargs={
    'mask_token' : tokenizer.mask_token
})
ds

Map:   0%|          | 0/14392 [00:00<?, ? examples/s]

Map:   0%|          | 0/3599 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'masked_text'],
        num_rows: 14392
    })
    test: Dataset({
        features: ['text', 'masked_text'],
        num_rows: 3599
    })
})

In [44]:
ds['train'][0]

{'text': "one person this i m going to kill myself and this is the end edit i ve taken half of the pill and i m not that drunk so i want to keep going edit had pill with coke and vodka before i felt like i needed to puke dry heaving and threw away the bottle and pill', 'hey everyone it s me ifiwasanotaku sorry that i ended up deleting it all last time but i gave it all some time and i don t think i can sustain this life anymore a i planned before i will be dying on th april but i just wanted to reach out to everyone who reached back to me that night and asked me to think some more i will",
 'masked_text': "one person this <mask> <mask> going to <mask> myself and this is <mask> end edit i ve taken half of the <mask> and i m not that <mask> so <mask> want <mask> keep going edit had <mask> with coke and vodka before i felt like i needed to <mask> dry heaving and threw away the bottle and <mask>','<mask> <mask> it s me ifiwasanotaku sorry that i ended up <mask> it all last <mask> but <mask

In [45]:
ds['test'][0]

{'text': 'not lecture time. At this point, I\'ve given up on mastering that kind of self control. I bought a timer lock so I can literally lock the candy away from myself when I shouldn\'t be eating it", \'Just let them be, let them come. Accepting and embracing. ERP and CBT is the key\', "Ya know, I was initially against the ban on putting a positive spin on ADHD but the more I think about it the more I think I support it. If *every* ADHD sub had the rule I think it would be an issue, but having one sub where you don\'t have to deal with that is nice.", "I think there are two good methods for this, although I definitely struggle with this: 1. Understand that',
 'masked_text': '<mask> lecture <mask>.At this point,I\'ve given up on mastering that <mask> <mask> self control.<mask> bought a timer lock <mask> I can literally lock the candy away from myself when I shouldn\'t be eating it",\'Just let them <mask>,let them come.<mask> and <mask>.ERP and CBT is the key\',"Ya know,I was initiall

In [46]:
tokenized_ds = ds.map(tokenize_function, batched=True, remove_columns=ds["train"].column_names)
tokenized_ds

Map:   0%|          | 0/14392 [00:00<?, ? examples/s]

Map:   0%|          | 0/3599 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14392
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3599
    })
})

## Model

In [47]:
model_path = "/kaggle/working/roberta-base-domain-adapted"  
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForMaskedLM.from_pretrained(model_path)
model.eval()

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNor

## Training

In [48]:
batch_size = 64 
model_name = model_checkpoint.split("/")[-1]
output_dir = f"{model_name}-domain-adapted-mental-dataset"

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    eval_strategy="epoch",
    learning_rate=5e-5,  
    weight_decay=0.01,
    num_train_epochs=6, 
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False, 
    fp16=True,  
    save_steps=1000,  
    logging_steps=len(tokenized_ds["train"]) // (batch_size * 2), 
    report_to="none", 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    # data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipykernel_19/4036430674.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,2.514000,2.274134
2,2.219900,2.170697
3,2.054200,2.147680
4,1.939200,2.044912
5,1.841000,2.043112
6,1.752700,2.042649


TrainOutput(global_step=1350, training_loss=2.0962455353913483, metrics={'train_runtime': 1451.3901, 'train_samples_per_second': 59.496, 'train_steps_per_second': 0.93, 'total_flos': 5683374821634048.0, 'train_loss': 2.0962455353913483, 'epoch': 6.0})

## Testing

In [49]:
eval_results = trainer.evaluate()
perplexity = math.exp(eval_results["eval_loss"])
print(f"Perplexity: {perplexity:.2f}")

trainer.save_model() 

Perplexity: 7.71


In [50]:
def compare_three_models(text, tokenizer, base_model, domain_model1, domain_model2):
    """
    Compare predictions of base BERT and two domain-adapted models on a masked text.
    """
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        base_logits = base_model(**inputs).logits
        domain1_logits = domain_model1(**inputs).logits
        domain2_logits = domain_model2(**inputs).logits

    # Locate [MASK] token index
    idx = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1][0]

    # Convert logits to probabilities
    base_probs = base_logits[0, idx].softmax(dim=0)
    domain1_probs = domain1_logits[0, idx].softmax(dim=0)
    domain2_probs = domain2_logits[0, idx].softmax(dim=0)

    def print_top(pred_probs, label):
        print(f"\n{label} predictions:")
        top = torch.topk(pred_probs, 5)
        for token_id, score in zip(top.indices, top.values):
            token = tokenizer.convert_ids_to_tokens(token_id.item())
            token = token.replace('Ġ', '')
            print(f"  {token:15s} | {score:.4f}")

    # Show results
    print("="*60)
    print(f"Input: {text}")
    print("="*60)

    print_top(base_probs, "Base Roberta")
    print_top(domain1_probs, "Domain Adapted 1")
    print_top(domain2_probs, "Domain Adapted 2")

    print("="*60)

In [51]:
base_model = AutoModelForMaskedLM.from_pretrained("roberta-base")
domain_adapted1 = AutoModelForMaskedLM.from_pretrained('/kaggle/working/roberta-base-domain-adapted')
domain_adapted2 = AutoModelForMaskedLM.from_pretrained("/kaggle/working/roberta-base-domain-adapted-mental-dataset")
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [52]:
texts = [
    f"I used to be able to {tokenizer.mask_token}.",
    f"I {tokenizer.mask_token} myself.",
    f"I have thoughts of {tokenizer.mask_token} myself."
]

for text in texts:
    compare_three_models(text, tokenizer, base_model, domain_adapted1, domain_adapted2)

Input: I used to be able to <mask>.

Base Roberta predictions:
  read            | 0.1334
  fly             | 0.1102
  drive           | 0.0911
  talk            | 0.0555
  write           | 0.0439

Domain Adapted 1 predictions:
  talk            | 0.1805
  read            | 0.1761
  write           | 0.0825
  drive           | 0.0699
  cook            | 0.0353

Domain Adapted 2 predictions:
  focus           | 0.3400
  talk            | 0.0420
  laugh           | 0.0336
  eat             | 0.0310
  think           | 0.0267
Input: I <mask> myself.

Base Roberta predictions:
  hate            | 0.1558
  hated           | 0.0901
  blame           | 0.0379
  forgive         | 0.0281
  love            | 0.0266

Domain Adapted 1 predictions:
  hate            | 0.2727
  love            | 0.0545
  blame           | 0.0523
  kill            | 0.0496
  hated           | 0.0316

Domain Adapted 2 predictions:
  hate            | 0.8290
  love            | 0.0674
  kill            | 0.0182
  blam

# Fine Tuning

## Dataset

In [53]:
ground_truth_df = pd.read_csv('/kaggle/input/erisk25/final-eriskt2-dataset-with-ground-truth/final-eriskt2-dataset-with-ground-truth/shuffled_ground_truth_labels.txt', sep=' ', header=None)
ground_truth_df.columns = ['user', 'is_depressed']
ground_truth_df.head()

,user,is_depressed
0,subject_BAPb4sn,0
1,subject_x12ArlU,0
2,subject_D1lB5nr,0
3,subject_aEK9WJt,0
4,subject_OpiNZAB,0


In [54]:
posts = {}
root_dir = '/kaggle/input/erisk25/final-eriskt2-dataset-with-ground-truth/final-eriskt2-dataset-with-ground-truth/all_combined'
for file in os.listdir(root_dir):
    file_path = os.path.join(root_dir, file)

    # user id
    user_id = file.split('.')[0]

    # reading the json file
    with open(file_path, 'r') as f:
        file_data = json.load(f)

    user_posts = []
    # using the file data
    for post in file_data:
        if post['submission']['target'] == True:
            user_posts.append(post['submission']['body'])

        comments = post['comments']
        for comment in comments:
            if comment['target'] == True:
                user_posts.append(comment['body'])

    posts[user_id] = user_posts

In [55]:
print(len(posts.keys()))

909


## Preprocessing

In [56]:
def preprocess(user_posts, N=35):
    # filter removed posts
    user_posts = list(filter(lambda x : x != '[removed]', user_posts))
    texts = ' '.join(user_posts)

    # lowercasing + removing punctuations and urls 
    cleaned = clean(texts, lower=False, no_urls=True, no_punct=False, replace_with_url='')
    
    # removing hashtags
    cleaned = re.sub(r'#\w+', '', cleaned)
    cleaned = ' '.join(cleaned.split())

    if len(cleaned) == 0:
        return []

    # convert into 35 segments
    segment_size = math.ceil(len(cleaned) / N)
    segments = [cleaned[i : i+segment_size] for i in range(0, len(cleaned), segment_size)]

    if len(segments) < N:
        segments += [""] * (N - len(segments))
    elif len(segments) > N:
        segments = segments[:N]

    return segments

In [57]:
processed_posts = {}
for user_id, user_post in posts.items():
    processed_posts[user_id] = preprocess(user_post)

len(processed_posts)

909

In [58]:
for user_id in processed_posts.keys():
    print(processed_posts[user_id])
    print(len(processed_posts[user_id]))
    break

["I'll start watching it! Curious to see what they got Heya! So I've been an Elemental Hero fan since I can remember. Own the same deck as on Master Duel. Now that I've completed it, I would like to venture into so", "me new fun themes and possibly find a new favorite deck. I've seen Dinomorphia and really like the design of the cards, but it seems like they don't got a lot going on! I have not watched the anime after GX but d", "id play a bit with tuners. Is there any anime I should watch that has really fun and cool cards and mechanic introduced? The decks I'm currently working on are Chaos, cyber dark and crystal beast. None of them ha", "ve the same spark as E-hero though) Any recommendations?? Thanks :3 I've never seen these before 🤣 very cute! I'll look into it! I love the purple haired character in Unchained!! Definitely gonna look into this, ", "thanks for the recommendation! Also swordsoul's chinese theme is fire! Might make this my synchro deck 🔥 I saw the art from Phantom Kn

In [59]:
train_ids, test_ids = train_test_split(ground_truth_df, test_size=0.1, stratify=ground_truth_df['is_depressed'], random_state=42)


train_users = train_ids['user'].values
test_users  = test_ids['user'].values

len(train_users), len(test_users)

(818, 91)

In [60]:
train_labels = ground_truth_df[ground_truth_df['user'].isin(train_users)]['is_depressed']
print(train_labels.value_counts())

test_labels = ground_truth_df[~ground_truth_df['user'].isin(train_users)]['is_depressed']
print(test_labels.value_counts())

is_depressed
0    726
1     92
Name: count, dtype: int64
is_depressed
0    81
1    10
Name: count, dtype: int64


In [61]:
def build_examples(selected_users):
    texts, labels, users = [], [], []
    for uid in selected_users:
        segments = processed_posts[uid]
        label = ground_truth_df[ground_truth_df['user'] == uid]['is_depressed'].values[0]
        for seg in segments:
            texts.append(seg)
            labels.append(label)
            users.append(uid)
    return texts, labels, users

In [62]:
train_texts, train_labels, train_user = build_examples(train_users)

len(train_texts), len(train_labels), len(train_user)

(28070, 28070, 28070)

In [63]:
train_texts[0], train_labels[0], train_user[0]

('Mozda najbolje da se smuvas sa tom ribom. Sve dok je ljubav iskrena, ja podrzavam. Moras da imas jedno gay iskustvo da bi znao da nisi gay. Svi smo kroz to prosli, opusteno. You wouldnt stone good ol Dave, would you now? >da li imam pravo na neku zalbu ili ne Ne. >jer su dve razlicite stete u pitanju, i da sam uspeo da sredim auto nakon prve stete, svakako bi u drugoj morali da mi isplate isti iznos , jer je trebao da de menja ceo branik... Ne. U ovom slucaju nema nikakvog "natezanja" i sve je cisto. Batici je ostecen 100% ispravan branik, dali su mu 14 soma da to prefarba. On je odlucio da je bolje 14 soma u dzepu i ogrebotina nego farbanje i 0 soma u dzepu, nije popravio branik. Kada mu se desila sledeca stvar, na istom braniku, da li je taj branik sada opet bio 100% ispravan? Pa nije imao je stetu, ogrebotinu. Koliko vredi ta ogrebotina? Onoliko koliko je placeno da se sanira, 14k, toliko su mu odbili do novog branika, jer je to vec dobio. Sve je vrlo cisto i po zakonu. To sto ti 

In [64]:
test_texts, test_labels, test_user = build_examples(test_users)
len(test_texts), len(test_labels), len(test_user)

(3115, 3115, 3115)

In [65]:
test_texts[0], test_labels[0], test_user[0]

("Itachi uchiha not that I am a huge fan but man his story hits hard 1. Naruto Shipudden / OG Naruto 2. Attack on Titan 3. Saiki 4. Berserk (manga) 5.Demon Slayer / jujutsu kaisen Hashirama hit pics I have watches 6 of them including Naruto both parts but I am in cbse Dead space 2 when I was a kid Very bad tier Come on guys you all know it's Itachi uchiha One piece and Naruto fans be hating you Real life oututsuki Go to a country where dental care is cheap like India and get your teeth fixed it will cost you less than usual high prices I can fix my teeth under 400 dollors Bro they all contain same amount of DNA so even any one of them have been fused you will be still you but XX or XY sperm have been fused you would have a different gender Naruto-kun Well guys we don't lose wars like america Mob psycho 100 is shit and boring Bro told the truth and don't lie we all know that ep 1071 was at peak hype and the hype died at 1072 ep WOW loyal fans Your Twitter was attacked by T-ELON missile 

In [66]:
train_dataset = Dataset.from_dict({
    "text": train_texts,
    "label": train_labels,
    "user": train_user
})
test_dataset = Dataset.from_dict({
    "text": test_texts,
    "label": test_labels,
    "user": test_user
})

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'user'],
        num_rows: 28070
    })
    test: Dataset({
        features: ['text', 'label', 'user'],
        num_rows: 3115
    })
})

In [67]:
split = dataset["train"].train_test_split(test_size=0.1, seed=42)

dataset = DatasetDict({
    "train": split["train"],
    "val": split["test"],
    "test": dataset["test"]
})

dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'user'],
        num_rows: 25263
    })
    val: Dataset({
        features: ['text', 'label', 'user'],
        num_rows: 2807
    })
    test: Dataset({
        features: ['text', 'label', 'user'],
        num_rows: 3115
    })
})

## Model

In [68]:
model_path = "/kaggle/working/roberta-base-domain-adapted-mental-dataset"

config = AutoConfig.from_pretrained(model_path)
config.num_labels = 2

model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    config=config,
)
print(model.classifier)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/roberta-base-domain-adapted-mental-dataset and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClassificationHead(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out_proj): Linear(in_features=768, out_features=2, bias=True)
)


In [69]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [70]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [71]:
tokenizer

RobertaTokenizerFast(name_or_path='/kaggle/working/roberta-base-domain-adapted-mental-dataset', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
)

In [72]:
def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

tokenized_dataset = dataset.map(tokenize_fn, batched=True)
tokenized_dataset

Map:   0%|          | 0/25263 [00:00<?, ? examples/s]

Map:   0%|          | 0/2807 [00:00<?, ? examples/s]

Map:   0%|          | 0/3115 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'user', 'input_ids', 'attention_mask'],
        num_rows: 25263
    })
    val: Dataset({
        features: ['text', 'label', 'user', 'input_ids', 'attention_mask'],
        num_rows: 2807
    })
    test: Dataset({
        features: ['text', 'label', 'user', 'input_ids', 'attention_mask'],
        num_rows: 3115
    })
})

## Training

In [73]:
train_labels = tokenized_dataset["train"]["label"]

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = torch.tensor(class_weights, dtype=torch.float)
print("Class weights:", class_weights)

Class weights: tensor([0.5647, 4.3632])


In [74]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    return {
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds, pos_label=1),
        'recall': recall_score(labels, preds, pos_label=1), 
        'f1': f1_score(labels, preds, pos_label=1)
    }

In [75]:
class WeightedTrainer(Trainer):
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights.to(logits.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [76]:
args = TrainingArguments(
    output_dir="./roberta-base-finetuned",
    overwrite_output_dir=True,
    save_total_limit=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=100,
    weight_decay=0.01,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",
    logging_dir="./logs",
)

trainer = WeightedTrainer(
    class_weights=class_weights,
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.469100,0.445583,0.754898,0.306709,0.886154,0.455696
2,0.374400,0.392966,0.838261,0.403875,0.833846,0.544177
3,0.312800,0.458771,0.915212,0.627566,0.658462,0.642643
4,0.235700,0.653922,0.896687,0.538631,0.750769,0.627249
5,0.192600,0.833457,0.920556,0.657407,0.655385,0.656394
6,0.133000,0.982474,0.889206,0.514894,0.744615,0.608805
7,0.092900,1.033684,0.900606,0.552752,0.741538,0.633377
8,0.080500,1.248836,0.922693,0.657895,0.692308,0.674663
9,0.070900,1.075411,0.892412,0.525054,0.741538,0.614796
10,0.056500,1.388727,0.918062,0.649842,0.633846,0.641745


TrainOutput(global_step=14220, training_loss=0.12789121076527526, metrics={'train_runtime': 5352.7819, 'train_samples_per_second': 471.96, 'train_steps_per_second': 14.759, 'total_flos': 2.991138566201856e+16, 'train_loss': 0.12789121076527526, 'epoch': 18.0})

## Testing

In [77]:
predictions = trainer.predict(tokenized_dataset["test"])

segment_preds = np.argmax(predictions.predictions, axis=1)
segment_labels = predictions.label_ids
user_ids = tokenized_dataset["test"]["user"]

df = pd.DataFrame({
    "user_id": user_ids,
    "segment_pred": segment_preds,
    "segment_label": segment_labels
})

def majority_vote(group):
    mode_result = stats.mode(group["segment_pred"], keepdims=True)
    return mode_result.mode[0]

user_preds_series = df.groupby("user_id").apply(majority_vote)
user_preds = user_preds_series.values
user_ids_unique = user_preds_series.index

user_labels_series = df.groupby("user_id")["segment_label"].first()
user_labels = user_labels_series.values

assert (user_preds_series.index == user_labels_series.index).all(), "User ID mismatch!"

accuracy = accuracy_score(user_labels, user_preds)
precision = precision_score(user_labels, user_preds, zero_division=0) 
recall = recall_score(user_labels, user_preds, zero_division=0)
f1 = f1_score(user_labels, user_preds, zero_division=0)

print(f"User-level Results:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

results_df = pd.DataFrame({
    "user_id": user_ids_unique,
    "true_label": user_labels,
    "pred_label": user_preds
})
results_df.to_csv("user_level_results.csv", index=False)

User-level Results:
Accuracy:  0.9551
Precision: 0.8750
Recall:    0.7000
F1 Score:  0.7778


/tmp/ipykernel_19/1365410557.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  user_preds_series = df.groupby("user_id").apply(majority_vote)
